In [3]:
import numpy as np
#from qiskit import *
EQ_TOLERANCE=1e-8
#* Determine whether 2 numbers are different from each other
#* if under tolerance, they are considered to be the same

In [4]:
def givens_matrix_elements(a, b, which='left'):
    if abs(a) < EQ_TOLERANCE:
        #* Handle case that a is zero
        cosine = 1.
        sine = 0.
        phase = 1.
    elif abs(b) < EQ_TOLERANCE:
        #* Handle case that b is zero and a is nonzero
        cosine = 0.
        sine = 1.
        phase = 1.
    # Handle case that a and b are both nonzero
    else:
        denominator = np.sqrt(abs(a)**2 + abs(b)**2)
        cosine = abs(b) / denominator
        sine = abs(a) / denominator
        sign_b = b / abs(b)
        sign_a = a / abs(a)
        phase = sign_a * sign_b.conjugate()
        # If phase is a real number, convert it to a float
        if np.isreal(phase):
            phase = np.real(phase)

    # Construct matrix and return
    if which == 'left':
        # We want to zero out a
        if (abs(np.imag(a)) < EQ_TOLERANCE and
                abs(np.imag(b)) < EQ_TOLERANCE):
            # a and b are real, so return a standard rotation matrix
            givens_rotation = np.array([[cosine, -phase * sine],
                                           [phase * sine, cosine]])
        else:
            givens_rotation = np.array([[cosine, -phase * sine],
                                           [sine, phase * cosine]])
    elif which == 'right':
        # We want to zero out b
        if (abs(np.imag(a)) < EQ_TOLERANCE and
                abs(np.imag(b)) < EQ_TOLERANCE):
            # a and b are real, so return a standard rotation matrix
            givens_rotation = np.array([[sine, phase * cosine],
                                           [-phase * cosine, sine]])
        else:
            givens_rotation = np.array([[sine, phase * cosine],
                                           [cosine, -phase * sine]])
    else:
        raise ValueError('"which" must be equal to "left" or "right".')
    return givens_rotation


def givens_rotate(operator, givens_rotation, i, j, which='row'):
    """Apply a Givens rotation to coordinates i and j of an operator."""
    if which == 'row':
        # Rotate rows i and j
        row_i = operator[i].copy()
        row_j = operator[j].copy()
        operator[i] = (givens_rotation[0, 0] * row_i +
                       givens_rotation[0, 1] * row_j)
        operator[j] = (givens_rotation[1, 0] * row_i +
                       givens_rotation[1, 1] * row_j)
    elif which == 'col':
        # Rotate columns i and j
        col_i = operator[:, i].copy()
        col_j = operator[:, j].copy()
        operator[:, i] = (givens_rotation[0, 0] * col_i +
                          givens_rotation[0, 1].conj() * col_j)
        operator[:, j] = (givens_rotation[1, 0] * col_i +
                          givens_rotation[1, 1].conj() * col_j)
    else:
        raise ValueError('"which" must be equal to "row" or "col".')

        
def givens_decomposition_square(unitary_matrix, always_insert=False):
    current_matrix = np.copy(unitary_matrix)
    n = current_matrix.shape[0]

    decomposition = []

    for k in range(2 * (n - 1) - 1):
        # Initialize the list of parallel operations to perform
        # in this iteration
        parallel_ops = []

        # Get the (row, column) indices of elements to zero out in parallel.
        if k < n - 1:
            start_row = 0
            start_column = n - 1 - k
        else:
            start_row = k - (n - 2)
            start_column = k - (n - 3)
        column_indices = range(start_column, n, 2)
        row_indices = range(start_row, start_row + len(column_indices))
        indices_to_zero_out = zip(row_indices, column_indices)

        for i, j in indices_to_zero_out:
            # Compute the Givens rotation to zero out the (i, j) element,
            # if needed
            right_element = current_matrix[i, j].conj()
            if always_insert or abs(right_element) > EQ_TOLERANCE:
                # We actually need to perform a Givens rotation
                left_element = current_matrix[i, j - 1].conj()
                givens_rotation = givens_matrix_elements(left_element,
                                                         right_element,
                                                         which='right')

                # Add the parameters to the list
                theta = np.arcsin(np.real(givens_rotation[1, 0]))
                phi = np.angle(givens_rotation[1, 1])
                parallel_ops.append((j - 1, j, theta, phi))

                # Update the matrix
                givens_rotate(current_matrix,
                              givens_rotation,
                              j - 1,
                              j,
                              which='col')

        # If the current list of parallel operations is not empty,
        # append it to the list,
        if parallel_ops:
            decomposition.append(tuple(parallel_ops))

    # Get the diagonal entries
    diagonal = current_matrix[range(n), range(n)]

    return decomposition, diagonal


In [5]:
nsites = 2
#* Parameters for setting up the Gaussian potential.
l_up = 4
m_up = 4.5
sigma_up = 1
#* For spin-dependent potentials

site_index = np.arange(1, nsites + 1)

spin_up_ham = np.diag([-1.] * (nsites - 1), k=1) + np.diag([-1.] * (nsites - 1), k=-1)
spin_up_ham += np.diag(-l_up * np.exp(-0.5 * ((site_index - m_up)**2 / sigma_up**2)))
print(spin_up_ham,"\n")
spin_down_ham = np.diag([-1.] * (nsites - 1), k=1) + np.diag([-1.] * (nsites - 1), k=-1)

rot_up, diag_up = givens_decomposition_square(spin_up_ham)
print("Parameters for the spin-up Givens sequence:\n", rot_up)

print("Diagonal elements:\n", diag_up)

print()

rot_down, diag_down = givens_decomposition_square(spin_down_ham)
print("Parameters for the spin-down Givens sequence:\n", rot_down)

print("Diagonal elements:\n", diag_down)

[[-0.00874996 -1.        ]
 [-1.         -0.17574773]] 

Parameters for the spin-up Givens sequence:
 [((0, 1, -1.5620465856164656, 0.0),)]
Diagonal elements:
 [-1.00003828  0.99842399]

Parameters for the spin-down Givens sequence:
 [((0, 1, -1.5707963267948966, 0.0),)]
Diagonal elements:
 [-1.  1.]
